# 自然数分割問題を計算する (拡張版)

- 自然数分割問題とは、ある自然数の集合を２つのグループA, Bに分割し、それぞれのグループ内の自然数の和が同じになるような分割方法を探す問題です。これをwildqatを使用して解いてみます。
- 今回は拡張版として、それぞれのグループに属す数字の数も等しいという条件を追加します　
- 例) 1, 2, 3 ,,,,


wildqatがインストールされていない場合は、環境に併せて以下のようにインストールしてください。



In [2]:
!pip3 install wildqat

     |████████████████████████████████| 13.9MB 12.4MB/s 
  Found existing installation: numpy 1.15.2
    Uninstalling numpy-1.15.2:
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/bin/f2py'
Consider using the `--user` option or check the permissions.



必要なライブラリをimportし、wildqatオブジェクトをインスタンス化します。

In [2]:
import wildqat as wq
import numpy as np
a = wq.opt()


解きたい問題のQUBOマトリクスを作成します。
N個の自然数の$i$番目の自然数を$n_i$とし、その自然数がどちらのグループに属するかを$q_i$で表します。
$n_i$がグループAに属する時には
$q_i=1$、グループBに属する時には$q_i=0$とします。
ここで、２つのグループ内のそれぞれの和が等しい時に最小となるようなコスト関数$E$を考えます。

この場合、

　$E=\{\sum_{i=1}^{N}n_i*(2q_i-1)\}^2$ 

とすれば、自然数$n_i$がグループＡに属するとき$2q_i-1=1$、グループBに属するとき$2q_i-1=-1$
になりますので、グループＡとグループＢに属する自然数の和が等しいときに
$E=0$になり、異なると$E>0$になります。

展開すると、

　$E=(\sum_{i=1}^{N}2n_iq_i)^2 -  2(\sum_{i=1}^{N}2n_iq_i)(\sum_{j=1}^{N}n_j) + (\sum_{i=1}^{N}n_i)^2$ 

コスト関数Eは最小化すれば良いので、最後の定数項は要らなくなります。またコスト関数は大きさのみ関係あるので、全体を４で割って

　$E=(\sum_{i=1}^{N}n_iq_i)^2 - \sum_{i=1}^{N}n_iq_i\sum_{j=1}^{N}n_j$ 

また、$q_i=1$または$q_i=0$のとき、$q_i^2=q_i$です。また、$\sum_{j=1}^N{n_j}$ はnの総和で定数ですので、
$n_{sum}$とします。さらに展開して整理すると</br>

　$E=\sum_{i=1}^{N}(n_i^2 - n_{sum}n_i)q_i +2 \sum_{i < j}n_in_jq_iq_j$ 

これを行列表記すると下記のようになります。

　$qubo = \left[\begin{array}{rrrrr}n_1^2 - n_{sum}n_1 & 2n_1n_2 & 2n_1n_3 & 2n_1n_4 & ...\\ 0 & n_2^2 - n_{sum}n_2 & 2n_2n_3& 2n_2n_4 &...\\ 0 & 0 & n_3^2 - n_{sum}n_3 & 2n_3n_4 & ...\\ 0 & 0 & 0 & n_4^2 - n_{sum}n_4 & ...\\ ... & ... & ... & ... &... \end{array} \right]$ 
 
---
**拡張問題を考える**
グループAとBに属する数字の個数の総和が等しい(個数の総和(A) = 個数の総和(B)) という条件を課すために、以下の式をコスト関数に付与する

　$E_1=\{\sum_{i=1}^{N}(2q_i-1)\}^2$ 

上述のように式変形し、ハイパーパラメータ $\alpha$,  と$\sum_{i=1} ^{N}1 = N$ を用いるとコスト関数は最終的に

　$E=\sum_{i=1}^{N}(n_i^2 - n_{sum}n_i)q_i +2 \sum_{i < j}n_in_jq_iq_j + \alpha \left[ \sum_{i=1}^{N}(1 - N)q_i +2 \sum_{i < j}q_iq_j \right]$
 
となる。

---

これをpythonプログラムで書き、シミュレータを実行して結果を得ます。

In [11]:
# ----- ハイパーパラメータ ----- 
alpha = 100
# ---------------------------

---
**複数回実行**

In [14]:
# numbers = np.array([3,2,6,9,2,5,7,3,3,6,7,3,5,3,2,2,2,6,8,4,6,3,3,6,4,3,3,2,2,5,8,9])
# numbers = np.array([3,2,6,9,2,5,7,3,3,6,7,3,5,3,2,2,2,6,8,4,6,3,3,6,4,3,3,2,2,5,8,9])
numbers = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
# numbers = np.array([1, 1, 1, 1, 2, 2])
# numbers = np.array([2, 5, 8, 6, 7, 2, 3, 10, 4, 11])

N = len(numbers)
a.qubo = np.zeros((numbers.size,numbers.size))
for i in range(numbers.size):
    for j in range(numbers.size):
        if i == j:
            a.qubo[i][i]=numbers[i]**2-numbers.sum()*numbers[i] + alpha*(1 - N) 
        elif i<j:
            a.qubo[i][j]=2*numbers[i] * numbers[j] + alpha*2

for _ in range(10):
    answer = np.array(a.sa())
    group1_string = ""
    group2_string = ""
    group1_sum = 0
    group2_sum = 0
    for i in range(numbers.size):
        if answer[i] == 0:
            group1_string+= '+' + str(numbers[i])
            group1_sum+=numbers[i]
        else:
            group2_string+= '+' + str(numbers[i])
            group2_sum+=numbers[i]

    print('------------------------------------------')
    print('numbers = ', numbers)
    print('answer  = ', answer)
    print((answer == 1).sum(), (answer==0).sum())
    print("A:", group1_string[1:],"=",group1_sum)
    print("B:", group2_string[1:],"=",group2_sum)
    E0 = (np.dot(numbers, answer))**2 - numbers.sum() * np.dot(numbers, answer)
    E1 = ( (answer.sum())**2 - N*answer.sum() ) * alpha
    E = E0 + E1
    print('E0   = ', E0)
    print("E1*a = ", E1)
    print('E    = ', E)

------------------------------------------
numbers =  [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
answer  =  [1 1 0 1 1 0 1 0 1 0 1 0 0 0]
7 7
A: 0+0+0+0+0+1+1 = 2
B: 0+0+0+0+0+0+0 = 0
E0   =  0
E1*a =  -4900
E    =  -4900
------------------------------------------
numbers =  [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
answer  =  [1 1 0 0 1 0 0 1 0 0 1 1 1 0]
7 7
A: 0+0+0+0+0+0+1 = 1
B: 0+0+0+0+0+0+1 = 1
E0   =  -1
E1*a =  -4900
E    =  -4901
------------------------------------------
numbers =  [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
answer  =  [1 1 1 1 1 0 0 1 0 1 0 0 0 0]
7 7
A: 0+0+0+0+0+1+1 = 2
B: 0+0+0+0+0+0+0 = 0
E0   =  0
E1*a =  -4900
E    =  -4900
------------------------------------------
numbers =  [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
answer  =  [0 0 1 0 1 1 1 1 0 1 0 1 0 0]
7 7
A: 0+0+0+0+0+1+1 = 2
B: 0+0+0+0+0+0+0 = 0
E0   =  0
E1*a =  -4900
E    =  -4900
------------------------------------------
numbers =  [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
answer  =  [1 0 1 0 1 0 0 1 0 0 1 1 1 0]
7 7
A: 0+0+0+0+0+0+1 = 1
B: 0+0+0+0+0+